In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [3]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yingshi_yvgao=pd.DataFrame(columns=columns)
channel_yingshi_yvgao

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [4]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [5]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[14]/div/a/span').click()  #选择频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[14]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'影视'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[5]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[5]/a').text
sub_channel

'预告·资讯'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [9]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yingshi_yvgao.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-25 19:31:23.459426


<ipython-input-9-c9ea10baecb9>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-c9ea10baecb9>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-c9ea10baecb9>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-c9ea10baecb9>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1CJ411V7aY视频属性的抓取


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.75)


In [14]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-14-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [10]:
channel_yingshi_yvgao.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85
1月,93,93,93,93,93,93,93,93,93,93,93,93,93,93,93,93,93,93,93,93
2月,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96
3月,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91
4月,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79
5月,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95
6月,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84
7月,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95
8月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97


In [14]:
channel_yingshi_yvgao.to_json('channel_yingshi_yvgao.json')

In [12]:
pd.set_option('display.max_rows', None)

In [13]:
channel_yingshi_yvgao[channel_yingshi_yvgao['时间']=='4月']

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1wK4y1k7rJ,4月,2020-04-16 15:14:36,翘tun工厂,无,729,75,《毒液2：屠杀》预告片,影视,预告·资讯,普通视频,...,"[超级英雄, 影视, 预告·资讯, 汤姆哈迪, 电影, 预告片, 毒液, VENOM]",01:31,99.1万,1.1万,269,1317,2678,820,585,否
1Ae411x7UD,4月,2020-04-01 20:16:57,c喋喋bb,无,473,42,这三人也太搞笑了哈哈哈哈哈看谁都像电灯泡 期待半是蜜糖半是伤,影视,预告·资讯,普通视频,...,"[半是蜜糖半是伤, 高瀚宇, 白鹿, 罗云熙, 搞笑, 影视, 预告·资讯]",09:56,93.1万,1.9万,562,8030,2467,993,2921,否
16p4y1C7oB,4月,2020-04-10 23:11:47,乌鸦预告片,个人认证,44.2万,1790,剧场版《鬼灭之刃·无限列车篇》正式预告，10月16日上映,影视,预告·资讯,普通视频,...,"[鬼灭之刃, 影视, 预告·资讯, 剧场版, 鬼灭之刃无限列车篇, 动画, 电影, 预告片,...",01:32,87.2万,1.6万,1015,3544,9999,1731,1239,否
1zK411L7tX,4月,2020-04-12 11:36:24,青呱瓷,无,190,57,【鬓边】花絮：尹正因为要多吃一个冰糖葫芦，彻底的崩溃了,影视,预告·资讯,普通视频,...,"[尹正, 黄晓明, 影视, 预告·资讯, 鬓边不是海棠红, 电视剧, 花絮, 商细蕊]",02:03,81.4万,2.2万,57,1857,204,411,334,否
1954y197kw,4月,2020-04-14 15:09:59,海鱼不知糖,无,276,67,余欢水：你叫辣么大声干嘛？,影视,预告·资讯,普通视频,...,"[搞笑, 影视, 预告·资讯, 电视剧, 剪辑]",00:35,65.0万,1011,41,278,19,74,86,否
1Kt4y127q3,4月,2020-04-13 10:39:23,乌鸦预告片,个人认证,44.2万,1790,电影版《我是大哥大》首曝预告，沙雕回来了！,影视,预告·资讯,普通视频,...,"[贺来贤人, 我是大哥大, 桥本环奈, 影视, 预告·资讯, 伊藤健太郎, 我是大哥大电影版...",01:32,64.2万,2.7万,3130,8038,2.5万,1752,1804,否
1VV411d7GU,4月,2020-04-29 11:15:51,李沁我老婆鸭,无,2.8万,482,【传闻中的陈芊芊】首爆预告！赵露思丁禹兮主演！,影视,预告·资讯,普通视频,...,"[丁禹兮, 赵露思, 影视, 预告·资讯, 古装, 电视剧, 爱情, 预告]",01:50,63.0万,1.7万,289,4264,4200,1382,860,否
1te411x7Jg,4月,2020-04-02 08:46:56,陆柒柒柒柒柒柒柒,无,313,116,【从结婚开始恋爱】【周雨彤×龚俊】没错，和赵简成亲的那个男人就是凌睿！事实证明 ，元仲辛并没...,影视,预告·资讯,活动作品,...,"[周雨彤, 影视, 预告·资讯, 龚俊, 电视剧, 从结婚开始恋爱, 甜宠, 全能打卡挑战]",01:45,56.3万,5582,43,832,129,376,251,否
1ug4y187pd,4月,2020-04-12 12:56:33,MARVEL粉丝团,无,379,3,我在拍复联4呢,影视,预告·资讯,普通视频,...,"[复仇者联盟4, 复仇者联盟, 漫威, 超级英雄, 影视, 预告·资讯, 电影, 花絮, 复联4]",00:16,55.9万,1.4万,112,2295,341,332,70,否
1HZ4y1j7aG,4月,2020-04-04 00:48:29,斑马春花蚂蚁阿金,无,234,83,【鬓边不是海棠红】这两货是想笑死我，然后继承我的花呗吗,影视,预告·资讯,普通视频,...,"[边江, 尹正, 影视, 预告·资讯, 鬓边不是海棠红, 程凤台, 商细蕊, 预告]",00:45,55.6万,4.3万,460,3167,247,261,1230,否


In [12]:
channel_shuma_shexiang

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1gJ411n7hp,1月,2020-01-12 16:31:27,甜咖啡后味带苦,无,723,16,中 门 对 狙,数码,摄影摄像,普通视频,...,"[摄影, 数码, 摄影摄像, 数码, 教程, 器材]",01:44,222.4万,9.7万,2.0万,1.4万,4980,3182,1571,否
1tJ41187w6,1月,2020-01-02 19:43:25,-LKs-,个人认证,165万,280,我找了11位知名UP主评价自己拍视频的设备丨器材终极选购指南,数码,摄影摄像,活动作品,...,"[相机, 数码, 摄影摄像, 数码, 何同学, 啊吗粽, 宝剑嫂, 自媒体, 大祥哥, 体验...",30:03,134.0万,10.7万,8.4万,5.8万,8907,3935,3.0万,否
1i7411i7yA,1月,2020-01-18 14:13:08,老蛙镜头,机构认证,2.6万,78,20秒的视频做了1个多月？揭秘模型世界的背后，准备好你的鸡皮疙瘩，51秒处别眨眼！——老蛙2...,数码,摄影摄像,普通视频,...,"[摄影, 短片, 数码, 摄影摄像, 数码, 创意, YOUTUBE, 镜头, 老蛙]",04:51,57.8万,4.4万,1.0万,1.6万,2043,641,983,否
1gJ411G7Cs,1月,2020-01-08 21:39:43,摄影师逝水,个人认证,0,84,一些实用且堪称神器的PS插件,数码,摄影摄像,普通视频,...,"[摄影, 数码, 摄影摄像, 数码, VLOG, 后期, 教程, ps, 器材]",06:01,42.3万,3.3万,3.4万,8.6万,2605,1405,612,否
1KJ411e7nN,1月,2020-01-02 20:00:26,TenGuSan,个人认证,52.9万,76,如何提高审美？审美有无高低？审美是谁建立？,数码,摄影摄像,普通视频,...,"[校园vlog, 相机, 脱口秀, 摄影, 公开课, vlog, 科普, 数码, 摄影摄像,...",27:08,39.6万,4.8万,5.4万,3.4万,6217,1156,1.0万,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1cr4y1w7ig,10月,2020-10-30 20:35:37,巡天者叶梓颐,个人认证,8.6万,65,困难重重，国家天文签约摄影师遭遇拍摄滑铁卢？！【华为Mate40 Pro】,数码,摄影摄像,活动作品,...,"[vlog, 数码, 摄影摄像, 数码, 放心购机攻略, 敦煌, 星空, 手机摄影, 华为,...",11:43,4.3万,3051,1391,692,255,337,349,否
1Y5411L7TW,10月,2020-10-22 21:20:39,深蓝摄影评测,无,6518,30,富士X-S10相机测试——2020 apsc画幅性价比最强机,数码,摄影摄像,活动作品,...,"[相机, 摄影, 数码, 摄影摄像, 数码, 数码玩家训练营, 拍照, 测评, xs10, ...",10:20,4.3万,773,219,221,111,273,104,否
1c541177hR,10月,2020-10-10 13:10:50,HusseinsDrama,个人认证,50.3万,279,索尼（Sony）12-24mm F2.8 GM深度评测，只要买得起，它就是一颗完美的镜头！,数码,摄影摄像,普通视频,...,"[摄影, 数码, 摄影摄像, 数码, 评测, 超广角大光圈, 摄像, 镜头, 数码评测, 超...",21:23,4.1万,1469,443,297,123,250,561,否
1DA41177UY,10月,2020-10-15 11:00:15,DJI大疆创新,机构认证,15.2万,495,大疆旗舰稳定器 - DJI RS 2 正式登场,数码,摄影摄像,普通视频,...,"[相机, 摄影, 数码, 摄影摄像, 数码, 科技, 测评, 器材, 大疆, DJI, 稳定器]",02:26,4.1万,1565,197,334,640,171,110,否
